In [1]:
# libraries
from dotenv import load_dotenv

# load environment variables from .env file
_ = load_dotenv("../../.env")

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    AIMessage,
    ChatMessage,
)

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    api_key="mistral",
    model="mistral",
    base_url="http://localhost:11434/v1",
    temperature=0,
)

class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [3]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [4]:
from langchain_core.pydantic_v1 import BaseModel


class Queries(BaseModel):
    queries: List[str]

In [5]:
from tavily import TavilyClient
import os

tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [45]:
def plan_node(state: AgentState):
    messages = [SystemMessage(content=PLAN_PROMPT), HumanMessage(content=state["task"])]
    response = model.invoke(messages)
    return {"plan": response.content}

def research_plan_node(state: AgentState):
    queries = model.invoke(
        [
            SystemMessage(content=RESEARCH_PLAN_PROMPT),
            HumanMessage(content=state["task"]),
        ]
    )
    content = state["content"] or []
    for q in queries.content.split("\n"):
        response = tavily.search(query=q[4:-1], max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


def generation_node(state: AgentState):
    content = "\n\n".join(state["content"] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    )
    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content)),
        user_message,
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1,
    }


def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"]),
    ]
    response = model.invoke(messages)
    return {"critique": response.content}


def research_critique_node(state: AgentState):
    queries = model.invoke(
        [
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state["critique"]),
        ]
    )
    content = state["content"] or []
    for q in queries.content.split("\n"):
        response = tavily.search(query=q[4:-1], max_results=2)
        for r in response["results"]:
            content.append(r["content"])
    return {"content": content}


def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [46]:
builder = StateGraph(AgentState)
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("planner")
builder.add_conditional_edges(
    "generate", should_continue, {END: END, "reflect": "reflect"}
)
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [47]:
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(checkpointer=checkpointer)
# graph = builder.compile()


In [48]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream(
    {
        "task": "what is the difference between langchain and langsmith",
        "max_revisions": 2,
        "revision_number": 1,
    },
    thread
):
    print("+++" * 10, f"start {list(s)[0]}", "+++" * 10)
    print(s[list(s)[0]][list(s[list(s)[0]])[0]])
    print("---" * 10, f"END {list(s)[0]}", "---" * 10)

++++++++++++++++++++++++++++++ start planner ++++++++++++++++++++++++++++++
 Title: A Comparative Analysis of LangChain and LangSmith: Understanding Their Unique Features and Differences

I. Introduction
   A. Brief overview of artificial intelligence, language models, and their importance in today's digital world
   B. Introduction to LangChain and LangSmith, highlighting their relevance in the field
   C. Statement of purpose: To compare and contrast LangChain and LangSmith, focusing on their unique features and differences

II. Background
   A. Detailed explanation of LangChain, its development, and key components
      1. Explanation of LangChain's capabilities, such as multilingual support, scalability, and real-time translation
      2. Discussion on the applications of LangChain in various industries (e.g., healthcare, finance, customer service)
   B. Detailed explanation of LangSmith, its development, and key components
      1. Explanation of LangSmith's capabilities, such as 

In [49]:
print(s["generate"]["draft"])

 VII. References
      A. List of credible sources used throughout the essay, including academic journals, industry reports, and official documentation from LangChain and LangSmith developers

This outline provides a comprehensive structure for your comparative analysis of LangChain and LangSmith. It covers their background, unique features, real-world applications, future implications, and concludes with a call to action for further exploration. By following this plan, you will create an informative and engaging essay that sheds light on the differences between these two AI language models. Good luck with your writing!


In [54]:
a = graph.get_state(thread).values
a

{'task': 'what is the difference between langchain and langsmith',
 'plan': " Title: A Comparative Analysis of LangChain and LangSmith: Understanding Their Unique Features and Differences\n\nI. Introduction\n   A. Brief overview of artificial intelligence, language models, and their importance in today's digital world\n   B. Introduction to LangChain and LangSmith, highlighting their relevance in the field\n   C. Statement of purpose: To compare and contrast LangChain and LangSmith, focusing on their unique features and differences\n\nII. Background\n   A. Detailed explanation of LangChain, its development, and key components\n      1. Explanation of LangChain's capabilities, such as multilingual support, scalability, and real-time translation\n      2. Discussion on the applications of LangChain in various industries (e.g., healthcare, finance, customer service)\n   B. Detailed explanation of LangSmith, its development, and key components\n      1. Explanation of LangSmith's capabilit

In [58]:
print(a["content"][-1])

AI revolutionizing industries worldwide: A comprehensive overview of its diverse applications - ScienceDirect AI revolutionizing industries worldwide: A comprehensive overview of its diverse applications Artificial Intelligence (AI) technology's rapid advancement has significantly changed various industries' operations. This comprehensive review paper aims to provide readers with a deep understanding of AI's applications & implementations, workings, and potential impacts across different sectors. Through extensive research on more than 200 research and many other sources, the authors have made every effort to present an accurate overview of the numerous applications of AI nowadays in industries such as agriculture, education, autonomous systems, healthcare, finance, entertainment, transportation, military, manufacturing, and more. No articles found. For all open access content, the relevant licensing terms apply.
